# [Graph Search, Shortest Paths, and Data Structures - Week4](https://www.coursera.org/learn/algorithms-graphs-data-structures/home/week/4)

- Hashing
- Universal Hashing
- Bloom Filters

SUGGESTED READINGS FOR WEEK 4: Algorithms Illuminated (Part 2), Chapter 12.

## XIV. HASHING: THE BASICS (Week 4)

### [Hash Tables: Operations and Applications](https://www.coursera.org/learn/algorithms-graphs-data-structures/lecture/b2Uee/hash-tables-operations-and-applications)

- hash table 能 support 的 operations 不多，但是卻做得很好。
- 可以把 hash table 當成 array
    - array 支援 random access，即我要拿第 $i$ 個 element
        - > Q: 但是 hash table??
- 但是 array 有些限制，例如我想存很多人的電話：
    - key 必須是「某個範圍內的」「integer」
    - 人名有太多種了，一次要 array 能容納所有可能的人名很困難
- 我們希望 array 有 reasonable size

#### Hash Table: Supported Operations

![](https://i.imgur.com/6i98lKm.png)
- 如果有 task 需要比較 (例如 maximum)，就不適合 hash table
- 雖然說 insert delete lookup 都可以在 constant time $O(1)$ 內達成，但是
    1. 很難 implement 得好，如果 implement 得不好那就不會是 constant time。
    2. worst case 其實並不是 constant time，只有 non-pathological(非病態?) dataset

#### Application: De-Duplication

![](https://i.imgur.com/rWylekk.png)
- **"stream"** 可以看下面幾個例子：
    1. 你要 scan 一個超大的 file，所以你 scan line by line。
    2. 會隨著時間一直收到 data，例如 router 會一直收到 data packet

#### Application: The 2-Sum Problem

![](https://i.imgur.com/Of2te2K.png)
- 最 naive 的做法就是用兩層 loop 跑
- 好一點的做法是：
    - 可以先 sort (想要比 $\theta(n^2)$ 好，可以先 sort ($\theta(n\log n)$) 看看能不能幫助 two-sum task。)
        - > **meta-knowledge: 想要比 某個複雜度 好，總是可以先做一些 (比該複雜度快的) pre-processing 看看能不能幫助該 task**
    - 然後跑每個數字 $x\in A$ 找 sorted array 裡面有沒有 $t-x$ (binary search 只需要 $\theta(\log n)$)
    - 這樣就只需要 $\theta(n\log n)$
- 再仔細想想，我們需要的其實是：對於每個數字 $x\in A$ 找 $A$ 裡面有沒有 $t-x$，**也就是我們只需要 lookup !!! 該用 hash table 了**
    - 所以只需要先 preprocess 成 hash table。(insert $n$ 次，$O(n)$)
    - 再對每個 $x$ 去 search $t-x$ 即可。(search $n$ 次，$O(n)$)
    - 所以總共只要 $O(n)$。

#### Further Immediate Applications

![](https://i.imgur.com/2horu0X.png)
- symbol tables: 需要檢查 variables 是否已經 defined。
- 把 blacklist 的 IP address 給 block 掉
- search algorithms，這邊說的是棋類遊戲的 configurations (配置；構造) search
    - 之前教的 BFS、DFS 都假設我們的 graph 可以直接存在 main memory
    - 但是這種棋類遊戲的 graph (vertices 是 configurations，edges 是 legal moves) 太大了，根本沒辦法記錄下來，因此沒辦法套 BFS 或 DFS。
    - 而我們還是想 explore 這個 graph，但又不想再遇到已經 explored 的，因為不想做 redundant work。
    - 這時候我們需要做的就是 lookup，因此 hash table 又派上用場啦~

### [Hash Tables: Implementation Details, Part I](https://www.coursera.org/learn/algorithms-graphs-data-structures/lecture/Ob0K7/hash-tables-implementation-details-part-i)

#### Hash Table: Supported Operations (Review)
- 回顧一下啊

#### High-Level Idea

![](https://i.imgur.com/fReK8Ap.png)
- universe 的 size 通常 超 級 大，例如所有可能的 names 大約有 $26^{30}$ 種
- 這邊的 reasonable size 可以是 thousands 到 millions，都比 universe 的 size 小多了
- 注意 evolving set $S$ 是可能變動的，不過我們先假設 $S$ 不會變動太多
- naive solution
    - array-based 佔用太多 space
    - list-based 花費太多 time

#### Quiz: Birthday Paradox

![](https://i.imgur.com/7pFCX8Y.png)
- 全都不 collide 的機率為 $\frac{365}{365}\times \frac{364}{365}\times \frac{363}{365}\times ...\times \frac{365-(n-1)}{365}=\frac{365\times ...\times (365-(n-1))}{365^n}$
- collide 的機率為 $1-\frac{365\times ...\times (365-(n-1))}{365^n}$
- 即使是 tiny dataset 而且 hash function 可以很 uniform 的分配到各個 bucket，仍然有很高機率會發生 collision。

In [7]:
def show_collide_prob(k):
    n = 365
    prod = 1
    for i in range(k):
        prod *= (n-i)/n
    prob = 1 - prod
    print('collide probability is %f for k = %d'%(prob, k))

show_collide_prob(23)
show_collide_prob(57)
show_collide_prob(367)


collide probability is 0.507297 for k = 23
collide probability is 0.990122 for k = 57
collide probability is 1.000000 for k = 367


#### Resolving Collisions

![](https://i.imgur.com/zTSqot0.png)
- Solution #1: chaining
- Solution #2: open addressing
    - 本來是 hash function $h(x)$，現在是 hash sequence $h_1(x),h_2(x),...$，例如：
        - linear probing: $h(x)$ 有人了就下一個 $h(x)+1$ ...
        - double hashing: ??? 是一直套同一個 hash function 嗎?
- 那麼 solution #1 和 #2 誰比較好? 各有優劣。
    - 若重視 space，就選 #2
    - 但是 #2 的 deletion 比較 tricky (難辦的@@)
- 接下來就要講「如何設計一個好的 hash function」

### [Hash Tables: Implementation Details, Part II](https://www.coursera.org/learn/algorithms-graphs-data-structures/lecture/1Y8t1/hash-tables-implementation-details-part-ii)

#### What Makes a Good Hash Function?

![](https://i.imgur.com/xd2qrnZ.png)


#### Bad Hash Functions

![](https://i.imgur.com/5El9Ok7.png)

#### Quick and Dirty Hash Functions

![](https://i.imgur.com/LZrPoQw.png)

### [Hash Tables: Implementation Details, Part II](https://www.coursera.org/learn/algorithms-graphs-data-structures/lecture/1Y8t1/hash-tables-implementation-details-part-ii)

## XV. UNIVERSAL HASHING (Week 4)

## XVI. BLOOM FILTERS (Week 4)